In [15]:
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import numpy as np

# 네이버 영화 댓글

- 댓글 한페이지
- 댓글 5페이지
- 댓글 전체페이지
- https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=1


## 댓글 한페이지

In [7]:
# 요청 => resposnse 객체 생성
url = ' https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=1'
res = requests.get(url)

# 요청확인
print(res.headers)
# print(f'res={res})

# response 객체 => 텍스트화
html_str = res.text
html_str

soup = BeautifulSoup(html_str, 'html.parser')
# soup

{'date': 'Fri, 19 Nov 2021 00:27:40 GMT', 'cache-control': 'no-store, no-cache, must-revalidate', 'set-cookie': 'JSESSIONID=A9BF9FF98E80232387131923741E44EE; Path=/', 'p3p': 'CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'content-length': '15333', 'content-type': 'text/html;charset=UTF-8', 'referrer-policy': 'unsafe-url', 'server': 'nfront'}


<td class="title">
<a href="/movie/bi/mi/basic.naver?code=167697" class="movie color_b">신과함께-인과 연</a>	
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:90%">별점 - 총 10점 중</span></span><em>9</em>
</div>
<br>스토리와 볼거리 모두 충족 가능 
<a href="#" onclick="report('poip****', 'FrOG/Fg1Ogmb/fygjVbTzDRwlu2aNezaOT6kza2Suus=', '스토리와 볼거리 모두 충족 가능', '17803660', 'point_after');" class="report" style="color:#8F8F8F" title="새 창">신고</a>
</td>

In [9]:
td_list = soup.select('.title')
# td_list = soup.find(class_ = 'title')
len(td_list)

10

In [10]:
td_list[0]

<td class="title">
<a class="movie color_b" href="/movie/bi/mi/basic.naver?code=167697">신과함께-인과 연</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:90%">별점 - 총 10점 중</span></span><em>9</em>
</div>
<br/>스토리와 볼거리 모두 충족 가능 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('poip****', 'zplJc/HH1f2UkDKCYzMDdsKOd6nb3teheqXECzMtooc=', '스토리와 볼거리 모두 충족 가능', '17803660', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>

In [16]:
# 제목, URL, 댓글내용, 평점
title = td_list[0].find('a').text
# url = td.list[0].find('a').get['href']
url = td_list[0].find('a').get('href')

# next_sibling을 이용한 요소탐색
# 다음 형제요소 탐색 : soup.태그.next_sibling.strip()
review = td_list[0].find('br').next_sibling.strip()
point = td_list[0].find('em').text
title, url, review, point

('신과함께-인과 연', '/movie/bi/mi/basic.naver?code=167697', '스토리와 볼거리 모두 충족 가능', '9')

In [18]:
# 2차원 리스트 => 데이터프레임
base_url = 'https://movie.naver.com'
temp_list = []
for row in td_list:
    title = td_list[0].find('a').text
    url = td_list[0].find('a').get('href')
    review = td_list[0].find('br').next_sibling.strip()
    point = td_list[0].find('em').text
    temp_list.append([title, url, review, point])
    
# 데이터프레임으로 저장
df_movie= pd.DataFrame(temp_list, columns=['title', 'url', 'review', 'point'])
df_movie

,title,url,review,point
0,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
1,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
2,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
3,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
4,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
5,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
6,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
7,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
8,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9
9,신과함께-인과 연,/movie/bi/mi/basic.naver?code=167697,스토리와 볼거리 모두 충족 가능,9


# 댓글 여러페이지

In [38]:
# 요청 => resposnse 객체 생성
url = ' https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page='
base_url = 'https://movie.naver.com'
temp_list = []
n = 5

for i in range(1,n+1):
    # 주소 생성
    url_page = url+str(i)
    print(url_page)
    
    res = requests.get(url_page)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    td_list = soup.select('.title')

    for row in td_list:
        title = row.find('a').text
        link = row.find('a')('href')
        review = row.find('br').next_sibling.strip()
        point = row.find('em').text
        temp_list.append([title, link, review, point])



 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=1
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=2
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=3
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=4
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=167697&target=after&page=5


In [39]:
# 데이터프레임으로 저장
df_movie2= pd.DataFrame(temp_list, columns=['title', 'link', 'review', 'point'])
df_movie2

,title,link,review,point
0,신과함께-인과 연,[],스토리와 볼거리 모두 충족 가능,9
1,신과함께-인과 연,[],외국에서 혼영하러 갔는데 한국인 나밖에 없어서 외국인들은 다 안 우는데 나 혼자 콧...,10
2,신과함께-인과 연,[],스케일크게 실사화는 잘된것같아요.,8
3,신과함께-인과 연,[],신과함께 1보다 2가 훨씬 재밌었음 근데 너무 길어 .... 죄와 벌은 7점 인...,8
4,신과함께-인과 연,[],감동적인 편이고 평타는 칩니다.,8
5,신과함께-인과 연,[],이 세대도 꾸준히 리메이크될 수 있을지 궁금,7
6,신과함께-인과 연,[],적당히 만든 신파 공산품,7
7,신과함께-인과 연,[],마동석은 미래를 내다보았다.박스피가 3000을 뚫었고 qqq는 연일 최고가 갱신이다...,10
8,신과함께-인과 연,[],재미있게 잘 봤어용.,8
9,신과함께-인과 연,[],1편에 비해 감동씬은 많지 않았지만 그냥 너무 잘 만든 느낌. 1편 보고 2편까지 ...,10


# 전체페이지 저장

In [ ]:
# 다음버튼이 없다면 마지막페이지
<a href="/movie/point/af/list.naver?st=mcode&amp;sword=13558&amp;target=after&amp;page=3"
title="다음" class="pg_next"><em>다음</em></a>

In [41]:
url = ' https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
base_url = 'https://movie.naver.com'
temp_list = []
page = 0

while True:
    # 주소생성
    page += 1
    url_page = url+str(page)
    print(url_page)
    
    
    res = requests.get(url_page)
    html_str = res.text
    soup = BeautifulSoup(html_str, 'html.parser')
    
    td_list = soup.select('.title')


    for row in td_list:
        title = row.find('a').text
        link = row.find('a')('href')
        review = row.find('br').next_sibling.strip()
        point = row.find('em').text
        temp_list.append([title, link, review, point])
        
    # 탈출조건 : 마지막 버튼이 없다면 
    next_btn = soup.select('.pg_next')
    if len(next_btn) == 0:
        break
    


 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=10
 https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=

In [42]:
# 데이터프레임으로 저장
df_movie3= pd.DataFrame(temp_list, columns=['title', 'link', 'review', 'point'])
df_movie3

,title,link,review,point
0,키드,[],짧고 굵게 행복하고 사랑스럽다,10
1,키드,[],(레미제라블의 장발장과 코제트간 관계가 다소간 롤리타적 색채를 띠는 걸 감안함)어쩌...,8
2,키드,[],과연 채플린의 영화이다! 이 참담한 포스트 코로나 시대에도 우리에게 웃음과 감동을 ...,10
3,키드,[],생에 첫 무성영화대사 없이 영상과 음악 만으로도 충분한 감동과 해학이 담겨 있다.찰...,10
4,키드,[],이탈리아 은행에서 50만불을 빌려 만든 휴머니스트 찰리 채플린의 장편 감독 데뷔작....,8
...,...,...,...,...
139,키드,[],신파도 맛깔스러운 채플린.,8
140,키드,[],그 당시의 시대적 상황을 웃음과 감동으로 잘 표현한 수작,10
141,키드,[],감동적인 드라마.,10
142,키드,[],웃음으로 포장한 가슴 아픈 비극,10


In [ ]:
# Selenium을 이용한 동적 컨텐츠 수집
원격으로 웹페이지 소스를 활용하기 위한 라이브러리
동적컨텐츠 수집에 주로 사용
작업환경 : 크롬 + 셀레니움용 웹드라이버 설치
주의사항 : 크롬 버전과 웹드라이버 버전이 일치
    
설치 과정

1) 크롬 웹브라우저 버전 확인 
 1시방향 옵션 버튼 클릭 후 [도움말]-[Chrome 정보] 확인 
 버전은? 예)94.0
2) 크롬 웹브라우저에 맞는 드라이버 파일 다운로드 
 https://chromedriver.chromium.org/downloads

 윈도우용인 경우 :   chromedriver_win32.zip

 쥬피터노트북 파일이 있는 폴더에 압축해제한다. 
 chromedriver.exe 파일이 있는지 확인 
3) 셀레니움 설치 
 pip list 로 selenium 이 있는지 확인
